In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import re
# import nltk
from sklearn.model_selection import train_test_split
# from nltk.corpus import stopwords
import joblib
import stopwordsiso as stopwords


In [2]:
df = pd.read_csv('../dataset/tagged.csv',  delimiter=',', quoting=2, encoding="utf-8")

In [3]:
df.head(5)

,sentences,sentiment_tag
0,Enjoying a beautiful day at the park! ...,2.0
1,Traffic was terrible this morning. ...,1.0
2,Just finished an amazing workout! 💪 ...,2.0
3,Excited about the upcoming weekend getaway! ...,2.0
4,Trying out a new recipe for dinner tonight. ...,0.0


In [4]:
stpwrd = stopwords.stopwords("en")

In [5]:
def procces_sentence(review:str)->str:
    sample_review = re.sub(r'http\S+', '', review)
    sample_review = re.sub("[^a-zA-Z]",' ',sample_review)
    sample_review = sample_review.lower()
    sample_review = sample_review.split()
    # swords = set(stopwords.words("english"))      
    swords = set(stpwrd)                                    
    sample_review = [w for w in sample_review if w not in swords and len(w) > 1]
    sample_review = " ".join(sample_review)
    return sample_review

def take_res(count:int)->int:
    # sub = df["Subjectivity"][count]
    pol = df["sentiment_tag"][count]
    return pol
    
    


In [6]:
from tqdm import tqdm

count_t = 219288

trainx = []
trainy = []
for i in tqdm(range(0,len(df["sentences"]))):
    trainx.append(procces_sentence(df["sentences"][i]))
    trainy.append(take_res(i))



  0%|          | 0/50732 [00:00<?, ?it/s]

100%|██████████| 50732/50732 [00:05<00:00, 8880.61it/s] 


In [7]:
trainy = np.array(trainy)
train_x, test_x, y_train, y_test = train_test_split(trainx,trainy, test_size = 0.1)
vectorizer = CountVectorizer( max_features = 15000 )
train_x = vectorizer.fit_transform(train_x)



In [8]:
test_xx = vectorizer.transform(test_x)
test_xx
test_xx = test_xx.toarray()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import joblib

# Sınıflandırma modelleri
models = {
    'Logistic Regressor': LogisticRegression(max_iter=1000),
    'Random Forest Classifier': RandomForestClassifier(),
    'Extra Trees Classifier': ExtraTreesClassifier(),
    'AdaBoost Classifier': AdaBoostClassifier(),
    'Decision Tree Classifier': DecisionTreeClassifier(),
    'K Neighbors Classifier': KNeighborsClassifier(),
    'CatBoost Classifier': CatBoostClassifier(learning_rate=0.1, depth=6, iterations=500, verbose=0),
    'XGBoost Classifier': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results_classification = {}

# Eğitim ve test
for name, model in models.items():
    model.fit(train_x, y_train)  # Y_train bir tek boyutlu sınıf etiketi vektörü olmalıdır
    y_pred = model.predict(test_xx)
    acc = accuracy_score(y_test, y_pred)
    results_classification[name] = {'Accuracy': acc}
    joblib.dump(model, name)
    print(f'{name}: Accuracy = {acc}')

# Sonuçları DataFrame'e dönüştürme
results_classification_df = pd.DataFrame(results_classification).T

print("Classification Results:")
print(results_classification_df)


Logistic Regressor: Accuracy = 0.8657863618446985
Random Forest Classifier: Accuracy = 0.8415451320457233
Extra Trees Classifier: Accuracy = 0.8624359479700433
AdaBoost Classifier: Accuracy = 0.6424911312573907
Decision Tree Classifier: Accuracy = 0.7049664958612535
K Neighbors Classifier: Accuracy = 0.5474970437524636
CatBoost Classifier: Accuracy = 0.8486401261332283


f:\code\My GitHub\sentence-analysis-updated\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:39:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Classifier: Accuracy = 0.4966495861253449
Classification Results:
                          Accuracy
Logistic Regressor        0.865786
Random Forest Classifier  0.841545
Extra Trees Classifier    0.862436
AdaBoost Classifier       0.642491
Decision Tree Classifier  0.704966
K Neighbors Classifier    0.547497
CatBoost Classifier       0.848640
XGBoost Classifier        0.496650


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier


model = CatBoostClassifier(learning_rate=0.1, depth=12, iterations=500, verbose=0)
model.fit(train_x, y_train)


In [16]:
new_sentence = "I had a wonderful experience at the park!"

processed_sentence = procces_sentence(new_sentence)

vectorized_sentence = vectorizer.transform([processed_sentence]).toarray()

prediction = model.predict(vectorized_sentence)
probabilities = model.predict_proba(vectorized_sentence)

print(f"Predicted Sentiment: {prediction}")
print(f"Probabilities: {probabilities}")


Predicted Sentiment: [2.]
Probabilities: [[0.4 0.  0.6]]


In [20]:
new_sentence = "I had a wonderful experience at the park!"

processed_sentence = procces_sentence(new_sentence)

vectorized_sentence = vectorizer.transform([processed_sentence]).toarray()

for name, model in models.items():
    prediction = model.predict(vectorized_sentence)
    probabilities = model.predict_proba(vectorized_sentence)

    # print(f"{name}: Predicted Sentiment: {prediction}")
    print(f"{name}: Probabilities: {probabilities}")

Logistic Regressor: Probabilities: [[0.49674539 0.06309844 0.44015616]]
Random Forest Classifier: Probabilities: [[0.26 0.02 0.72]]
Extra Trees Classifier: Probabilities: [[0.19 0.01 0.8 ]]
AdaBoost Classifier: Probabilities: [[0.31902836 0.33740336 0.34356828]]
Decision Tree Classifier: Probabilities: [[0. 0. 1.]]
K Neighbors Classifier: Probabilities: [[0.4 0.  0.6]]
CatBoost Classifier: Probabilities: [[0.2071097  0.17276459 0.6201257 ]]
XGBoost Classifier: Probabilities: [[0.0530788  0.6142782  0.33264294]]
